In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from metaflow import Flow, get_metadata
from matplotlib import pyplot as plt

import plotly
from math import floor
import pickle
import plotly.graph_objects as go
from scipy.signal import savgol_filter

np.set_printoptions(suppress=True)

In [3]:
def plot_spread(array, title, xlabel, ylabel):
    mean = smooth(np.mean(array, 0), 20)
    std = smooth(np.std(array, 0), 20)

    fig = go.Figure()

    colors = plotly.colors.sequential.Magenta

    upper_bound = go.Scatter(x=list(range(len(mean))), 
                             y=mean+std,
                             mode='lines',
                             fillcolor='rgba(0, 138, 196, 0.1)',
                             fill='tonexty',
                             line=dict(width=0),
                             hoverinfo='skip')

    trace = go.Scatter(x=list(range(len(mean))), 
                             y=mean,
                             mode='lines',
                             fillcolor='rgba(0, 138, 196, 0.1)',
                             fill='tonexty',
                             line=dict(color='#008ac4', width=2))

    lower_bound = go.Scatter(x=list(range(len(mean))), 
                             y=mean-std,
                             mode='lines',
                             line=dict(width=0),
                             hoverinfo='skip')

    data = [lower_bound, trace, upper_bound]

    layout = go.Layout(
        title_x=0.5,
        xaxis=dict(title=xlabel),
        yaxis=dict(title=ylabel),
        title=title,
        showlegend = False)

    fig = go.Figure(data=data, layout=layout)

    return fig

In [20]:
def smooth(y, factor):
    box = np.ones(factor)/factor
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth[(factor//2):-(factor//2)]

def plot_runs(log_file='logs', train_test='train'):
    with open(f'data/{log_file}.pkl', 'rb') as handle:
        logs = pickle.load(handle)
        
    fig = go.Figure()
    
    hold       = smooth(logs['baseline'], 4)
    portfolios = logs['portfolio']

    for run, portfolio in enumerate(portfolios):
        scale = run / len(portfolios)
        
        portfolio = smooth(portfolio, 4)
        
        fig.add_trace(go.Scatter(x=list(range(len(portfolio))), 
                                 y=portfolio,
                                 mode='lines',
                                 line=dict(color=f'rgba(0, 138, 196, {scale + .2})'),
                                 name=f'Run {run+1}'))
        
    fig.add_trace(go.Scatter(x=list(range(len(hold))), 
                             y=hold,
                             mode='lines',
                             line=dict(color='#ff8a00', width=3),
                             name=f'Original'))
    
    fig.update_layout(
        xaxis=dict(title='Step'),
        yaxis=dict(title='Portfolio value ($)'),
        title='Portfolio value of different runs',
        title_x=0.5,
        showlegend = False)
    
    fig.show()
    
    fig = plot_spread(np.array(logs['portfolio']), 
            title='Spread of different runs', 
            xlabel='Step', 
            ylabel='Portfolio value ($)')
    
    fig.add_trace(go.Scatter(x=list(range(len(hold))), 
                             y=smooth(logs['baseline'], 20),
                             mode='lines',
                             line=dict(color='#ff8a00', width=3),
                             name=f'Original'))
    
    fig.show()
    
plot_runs('logs')

In [22]:
plot_runs('logs')